In [123]:
import py_vncorenlp
import os
original_cwd = os.getcwd()
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos", "ner", "parse"], save_dir=os.path.join(original_cwd, "vncorenlp"))
os.chdir(original_cwd)

In [22]:
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = set(line.strip().lower() for line in f if line.strip())
stopwords.add('sto')

In [18]:
from gensim.models import Word2Vec


word2vec_model = Word2Vec.load("word2vec_vi_bao_st.model")

In [ ]:
def most_similar_stopword(word, model, topn=10):
    if word in model.wv:
        similar_words = model.wv.most_similar(word, topn=topn)
        return [(w.replace('_', ' '), score) for w, score in similar_words]
    else:
        return []

In [60]:
import kenlm

kenlm_model = kenlm.Model("vi_model_6gramVinToken.binary")

In [48]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/gpt2-vietnamese", cache_dir="./transformers_cache")
model = AutoModelForCausalLM.from_pretrained("NlpHUST/gpt2-vietnamese", cache_dir="./transformers_cache")

tokenizer_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [49]:
with open("train_corpus_vinAI.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
lines = [line.strip() for line in lines if line.strip()]

In [53]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": lines})


block_size = 128  # or 256 / 512 depending on GPU memory

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=block_size)

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],  # make sure only 'input_ids' remains
)

# 1. Concatenate all tokenized input_ids into one long list
def group_texts(examples):
    concatenated_ids = []
    concatenated_masks = []

    for ids, masks in zip(examples["input_ids"], examples["attention_mask"]):
        concatenated_ids.extend(ids)
        concatenated_masks.extend(masks)

    total_length = (len(concatenated_ids) // block_size) * block_size

    input_ids = [concatenated_ids[i:i+block_size] for i in range(0, total_length, block_size)]
    attention_mask = [concatenated_masks[i:i+block_size] for i in range(0, total_length, block_size)]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": input_ids.copy()
    }


# 2. Now map it with batched=True over the whole dataset
lm_dataset = tokenized_dataset.map(group_texts, batched=True, batch_size=1000)





Map:   0%|          | 0/72694 [00:00<?, ? examples/s]

Map:   0%|          | 0/72694 [00:00<?, ? examples/s]

In [54]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gpt2-vietnamese-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    warmup_steps=100,
    weight_decay=0.01,
    fp16=True,  # set to False if you're not using GPU with mixed precision
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    tokenizer=tokenizer,
)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_24996\3912580168.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [56]:
# Fix: Add pad token if missing
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [57]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.375400
200,3.014200
300,2.989500
400,2.853200
500,2.883800
600,2.816100
700,2.695800
800,2.744500
900,2.664300
1000,2.641100


c:\Python310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Python310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=35502, training_loss=2.230347095049708, metrics={'train_runtime': 57530.779, 'train_samples_per_second': 1.234, 'train_steps_per_second': 0.617, 'total_flos': 4637998891008000.0, 'train_loss': 2.230347095049708, 'epoch': 3.0})

In [63]:
import os, re

root = r"C:\Users\Lenovo\Desktop\work\ctu\NLP\nlp-ctu\gpt2-vietnamese-finetuned"
final_dir = os.path.join(root, "final")
os.makedirs(final_dir, exist_ok=True)

# save both model and tokenizer together
model.save_pretrained(final_dir)
tokenizer.save_pretrained(final_dir)
print("Saved to:", final_dir)


Saved to: C:\Users\Lenovo\Desktop\work\ctu\NLP\nlp-ctu\gpt2-vietnamese-finetuned\final


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tok = AutoTokenizer.from_pretrained(r"gpt2-vietnamese-finetuned\final")
mdl = AutoModelForCausalLM.from_pretrained(r"gpt2-vietnamese-finetuned\final")


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
mdl.to(device)

In [ ]:
def generate_vietnamese_sentences(prompt, model, tokenizer, device, rdrsegmenter, do_sample=True, top_p=0.9, top_k=50, temperature=0.8):
    prompt = prompt.strip()
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    max_new_tokens = prompt.count(' ') * 2

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=top_p,
            top_k=top_k,
            temperature=temperature,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            num_return_sequences=10
        )

    prompt_segmented = rdrsegmenter.word_segment(prompt)[0].split()
    result = []
    for output_id in output_ids:
        output_text = tokenizer.decode(output_id, skip_special_tokens=True)
        output_text = re.sub(r'\s+', ' ', output_text).strip()
        
        segmented_text = rdrsegmenter.word_segment(output_text)[0].split()

        for i in range(0, 3):
            segmented_text_cropped = segmented_text[:len(prompt_segmented) + i]

            if segmented_text_cropped != prompt_segmented and segmented_text_cropped not in result:
                result.append(segmented_text_cropped)


    return [' '.join(item).replace('_', ' ') for item in result]

In [159]:
generate_vietnamese_sentences("tai nạn giao ", mdl, tok, device, rdrsegmenter, do_sample=True, top_p=0.9, top_k=50, temperature=0.8)

['tai nạn giao thông',
 'tai nạn giao thông xảy',
 'tai nạn giao thông xảy địa bàn',
 'tai nạn giao thông chết',
 'tai nạn giao thông chết bị thương',
 'tai nạn giao thông nghiêm trọng',
 'tai nạn giao thông nghiêm trọng vụ',
 'tai nạn giao thông làm',
 'tai nạn giao thông làm chết',
 'tai nạn giao thông vụ',
 'tai nạn giao thông vụ tai nạn',
 'tai nạn giao thông xảy vụ',
 'tai nạn giao thông xảy đoạn']